In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
os.chdir('C://work/dev/python/progs/texts/sec_bert/')

In [26]:
import pandas as pd 
import joblib


In [27]:
DN = 'data/reports_attck_tables/2022'
fns = [f'{DN}/{it}' for it in os.listdir(DN) if 'json' in it]

tab_df = pd.concat([pd.read_json(fn).explode('tables') for fn in fns], ignore_index=True)

tab_df['tables'] = tab_df['tables'].map(lambda x: [(it1, it2) 
                                for it1, it2 in zip(x['TechniqueID'].values(), x['Procedure'].values())])
tab_df = tab_df.explode('tables').drop_duplicates().reset_index(drop=True)
tab_df['technic'] = tab_df['tables'].map(lambda x: x[0])
tab_df['sentence'] = tab_df['tables'].map(lambda x: x[1])

tab_df = tab_df[tab_df['sentence'].str.split().str.len()>3]
tab_df['labels'] = tab_df['technic'].map(lambda x: [x.upper()])
tab_df.head()

,report_path,tables,technic,sentence,labels
0,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1583.001, Mustang Panda has registered domai...",t1583.001,Mustang Panda has registered domains for use a...,[T1583.001]
1,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1583.003, Some download servers used by Must...",t1583.003,Some download servers used by Mustang Panda ap...,[T1583.003]
2,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1583.004, Mustang Panda uses servers that ap...",t1583.004,Mustang Panda uses servers that appear to be e...,[T1583.004]
3,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1587.001, Mustang Panda has developed custom...",t1587.001,Mustang Panda has developed custom loader and ...,[T1587.001]
4,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1588.006, Multiple DLL hijacking vulnerabili...",t1588.006,Multiple DLL hijacking vulnerabilities are use...,[T1588.006]


In [36]:
tab_df[tab_df['technic']=="t1574.002"]

,report_path,tables,technic,sentence,labels
10,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, The downloader obtains and launche...",t1574.002,The downloader obtains and launches a vulnerab...,[T1574.002]
56,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, Hijack Execution Flow: DLL Side-Lo...",t1574.002,Hijack Execution Flow: DLL Side-Loading,[T1574.002]
191,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, FlowCloud uses DLL Side-Loading to...",t1574.002,FlowCloud uses DLL Side-Loading to launch its ...,[T1574.002]
664,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, Hijack Execution Flow: DLL Side-Lo...",t1574.002,Hijack Execution Flow: DLL Side-Loading,[T1574.002]
681,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, SpyAgent hooks and patches various...",t1574.002,SpyAgent hooks and patches various API functio...,[T1574.002]
762,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, Many of the Lazarus droppers and l...",t1574.002,Many of the Lazarus droppers and loaders use a...,[T1574.002]
928,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, MirrorFace side-loads LODEINFO by ...",t1574.002,MirrorFace side-loads LODEINFO by dropping a m...,[T1574.002]
953,/opt/nifi/configuration_resources/rstthreat/ti...,"(t1574.002, The attackers abused the legitimat...",t1574.002,The attackers abused the legitimate Logagent.e...,[T1574.002]


In [7]:
label2tactic = joblib.load('data/label2tactic.pkl')

# тут и мобильные угрозы есть, поэтому фильтруем их
tab_df = tab_df[tab_df['labels'].map(lambda x: all([it in label2tactic for it in x]))]

In [8]:
tab_df.shape

(837, 5)

In [9]:
import joblib
joblib.dump(tab_df, 'data/rep_data.pkl')

['data/rep_data.pkl']

# Аномалии

In [15]:
tab_df = tab_df[tab_df['sentence'].str.split().str.len()<3]

# Проблемы 

- есть дубли, случайно или что-то перезаписано (файл attck_procedures_202203, 0 элемента, 2 таблицы)
- короткие описания в пару слов, могут вводить в заблуждение модель (файл attck_procedures_202203, 1 элемент, 0 таблица)

# Старое

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)



In [19]:
import os
dn = 'data/reports_202212'
fns = [f'{dn}/{it}/model.json' for it in os.listdir(dn)]
# fn = fns[5]
fn

'data/reports_202212/20221202_decoded_avast_io_report_0x2ca02e2c/model.json'

In [18]:
import json

def get_report_paras(fn):
    rep_sents = [it for chapter in json.load(open(fn, 'rt'))['chapters'] for it in chapter['sentences']]
    rep_sents = [it['text'] for it in rep_sents]
    return rep_sents
    
rep_sents = get_report_paras(fn)

FileNotFoundError: [Errno 2] No such file or directory: 'data/reports_202212/20221202_decoded_avast_io_report_0x2ca02e2c/model.json'

In [20]:
import re
regexp_ttp = re.compile(r'(^| )(t\d{3,}(\.\d+)?)([., ]|$)', flags=re.IGNORECASE)
# regexp_ttp = r'[\d\.]{1,}'
for fn in fns:
    if os.path.exists(fn):
        rep_sents = get_report_paras(fn)
        text = '\n'.join(rep_sents)
        res_l = re.findall(regexp_ttp, text)
        if len(res_l)>0:
            print(fn)

data/reports_202212/20221202_asec_ahnlab_com_report_0x97bb2408/model.json
data/reports_202212/20221205_us-cert_cisa_gov_report_0xdc5d404f/model.json
data/reports_202212/20221223_trendmicro_com_report_0x203a8843/model.json


In [28]:
fn = 'data/reports_202212/20221223_trendmicro_com_report_0x203a8843/model.json'
rep_sents = get_report_paras(fn)
text = '\n'.join(rep_sents)
# print(text)
res_l = re.findall(regexp_ttp, text)
res_l

[(' ', 'T1218.007', '.007', ' '),
 (' ', 'T1218.011', '.011', ' '),
 (' ', 'T1218.011', '.011', ' '),
 (' ', 'T1027.009', '.009', ' '),
 (' ', 'T1218.011', '.011', ' ')]

In [ ]:
для всех sentences из каждой chapter



In [ ]:
поисковой механизм
- берем поабзацно и сравниваем